In [1]:
# load the library
import numpy as np
import pandas as pd
from tqdm import tqdm

# load selenium
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pyperclip
from IPython.display import clear_output
import sqlite3
from pandas.io import sql
import datetime

In [2]:
# handling page refresh
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [38]:
# Set name of your SQLite database
db_name = "freight"

sqlite_db = db_name + '.sqlite'
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

In [ ]:
lane_header = pd.read_excel('Lane Title.xlsx')
lane_header = list(lane_header.columns)

In [39]:
# load the library
import requests
import bs4
from bs4 import BeautifulSoup
import numpy as np
import requests
import pandas as pd
from tqdm import tqdm
pd.set_option('max_colwidth',1000)
import time
from sklearn.externals import joblib

In [40]:
# download chromedriver.exe and save to a local drive locaiton:
location_of_chromedriver = "Driver/chromedriver.exe"
site_url = "https://www.bigschedules.com/routes"

In [41]:
# set browser and go to google maps
def init_browser(URL = 'https://www.google.com.hk/maps'):
    browser = webdriver.Chrome(location_of_chromedriver)
    browser.get(URL)
    return browser

In [42]:
# open table management

def get_tables(otm, row_no=6):
    time.sleep(5)
    html = otm.page_source
    data_df_day = pd.DataFrame()
    data_df_direct = pd.DataFrame()
    
    bs=BeautifulSoup(html, "lxml")
    rows = bs.find_all('tr')
    tbl_day = []
    tbl_direct = []
    tbl_result = []
    for row in rows:
        cols=row.find_all('td')
        colstrip=[x.text.strip() for x in cols]
    #     print(len(colstrip))
        tbl_result.append(colstrip)
    
#     get_lst = []
    for i in range(len(tbl_result)):
        if len(tbl_result[i]) == row_no:
            if "day" in tbl_result[i][0]:
                tbl_day.append(tbl_result[i])
                tbl_direct.append(tbl_result[i+1])
    
    df_tmp_day = pd.DataFrame(tbl_day)
    df_tmp_direct = pd.DataFrame(tbl_direct)
    
    data_df_day = data_df_day.append(df_tmp_day)
    data_df_direct = data_df_direct.append(df_tmp_direct)
    
    result = pd.concat([data_df_day, data_df_direct], axis=1, join='inner')
    
    return result

In [43]:
def keep_trying(function_loop = get_tables, max_run=10):
    counter = 0
    found_it = False
    while found_it == False and counter < max_run:
        try:
            function_loop()
            found_it = True
        except:
            time.sleep(2)
        counter = counter + 1

In [47]:
def scrape_actions(orig_in_txt = 'Shanghai', dest_in_txt = 'Southampton'):

    rate_site.refresh()


    rate_site.switch_to_default_content()
#     iframe = rate_site.find_elements_by_tag_name('iframe')
#     rate_site.switch_to_frame(iframe[0])

    time.sleep(1)
    orig_txt = rate_site.find_elements_by_id('targetOriginal')
    dest_txt = rate_site.find_elements_by_id('targetDestination')


    orig_txt[0].send_keys(orig_in_txt)
    time.sleep(3)
    orig_txt[0].send_keys(Keys.RETURN)

    dest_txt[0].clear()
    dest_txt[0].send_keys(dest_in_txt)
    time.sleep(3)
    dest_txt[0].send_keys(Keys.RETURN)
    
    search_btn = rate_site.find_element_by_id('main_a_search')
    search_btn.click()

    time.sleep(5)
    rate_tables = get_tables(rate_site)
    

    return rate_tables

In [17]:
rate_site = init_browser(site_url)

In [83]:
freight_lanes = pd.read_excel('country_list.xlsx')

In [84]:
freight_lanes.shape

(6, 2)

In [85]:
lane_result = pd.DataFrame()
for i in tqdm(range(len(freight_lanes))):
    orig_in_txt = freight_lanes['Orig'].iloc[i]
    dest_in_txt = freight_lanes['Dest'].iloc[i]
    
    try:
        lane_tmp = scrape_actions(orig_in_txt, dest_in_txt)
        lane_result = lane_result.append(lane_tmp)
        lane_tmp.columns = lane_header
        lane_tmp.to_sql('lanes', index=False, if_exists='append' , con=conn)
    except:
        pass

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:56<00:00,  9.35s/it]


In [65]:
lane_result = pd.read_excel('lane_result.xlsx')

In [66]:
lane_result.columns = [j for j in range(len(list(lane_result.columns) ))]
lane_result.to_sql('lanes', index=False, if_exists='append' , con=conn)

In [79]:
lane_result = pd.read_sql('select * from lanes', con=conn)

In [82]:
lane_result.to_excel('lane_result.xlsx', index=False)

In [86]:
conn.close()